<a href="https://colab.research.google.com/github/speakerv/Testing/blob/master/m5_SIA_model_v9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import gc
import warnings

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

from scipy.optimize import minimize
from IPython.display import Audio

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)
register_matplotlib_converters()
sns.set()

In [0]:
import requests, zipfile, io
#The copied URL goes here ->
r = requests.get( 'https://github.com/speakerv/Testing/blob/master/M5.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [0]:
import IPython

def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

In [0]:
def on_kaggle():
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ

In [0]:
if on_kaggle():
    os.system("pip install --quiet mlflow_extend")

In [0]:
def reduce_mem_usage(df, verbose=False):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    int_columns = df.select_dtypes(include=["int"]).columns
    float_columns = df.select_dtypes(include=["float"]).columns

    for col in int_columns:
        df[col] = pd.to_numeric(df[col], downcast="integer")

    for col in float_columns:
        df[col] = pd.to_numeric(df[col], downcast="float")

    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [0]:
def read_data():
    INPUT_DIR = "/kaggle/input" if on_kaggle() else "input"
    INPUT_DIR = f"{INPUT_DIR}/m5-forecasting-accuracy"

    print("Reading files...")

    calendar = pd.read_csv(f"{INPUT_DIR}/calendar.csv").pipe(reduce_mem_usage)
    prices = pd.read_csv(f"{INPUT_DIR}/sell_prices.csv").pipe(reduce_mem_usage)

    sales = pd.read_csv(f"{INPUT_DIR}/sales_train_validation.csv",).pipe(
        reduce_mem_usage
    )
    submission = pd.read_csv(f"{INPUT_DIR}/sample_submission.csv").pipe(
        reduce_mem_usage
    )

    print("sales shape:", sales.shape)
    print("prices shape:", prices.shape)
    print("calendar shape:", calendar.shape)
    print("submission shape:", submission.shape)

    # calendar shape: (1969, 14)
    # sell_prices shape: (6841121, 4)
    # sales_train_val shape: (30490, 1919)
    # submission shape: (60980, 29)

    return sales, prices, calendar, submission

In [0]:
# type(train_data)
# train_data['Value'][10]

# temp_data=train_data.drop(index=np.arange(0,10,1).tolist()) 

# print(temp_data)
# print(train_data)



In [29]:
# INPUT_DIR = "output/kaggle/working"
# train_data=sales.loc[0][6:].to_frame()
# train_data.columns=['Value']
# train_data
# train_data.to_csv("test1_train_data.csv",index=True)

train_data=pd.read_csv("/content/test1_train_data.csv")
train_data['Value']

# plt.plot(train_data.index,train_data['Value'])
# fig_size = plt.rcParams["figure.figsize"]
# fig_size[0] = 500
# fig_size[1] = 500
# plt.rcParams["figure.figsize"] = fig_size
# train_data.shape[0]

fig = go.Figure()
# x = np.arange(len(prediction))
fig.add_trace(go.Scatter(x=train_data.index.values,
                         y=train_data['Value'],
                         name ='pre_interval')) 
fig.show()

num_for_zeros_window=10
counter_zeros_in_sequence = 0 
flag_start_count=False
for i in train_data.index.values:
      
    if train_data['Value'][i] != 0 and flag_start_count:
        flag_start_count = False 
        if counter_zeros_in_sequence >= num_for_zeros_window:
#             print('point 4')
            train_data=train_data.drop(index=np.arange(left_index,right_index).tolist())                   
                       
    
    if train_data['Value'][i] == 0 and flag_start_count:
#         print('point 2')
        counter_zeros_in_sequence += 1
        right_index=i
        
    
    if train_data['Value'][i] == 0 and not(flag_start_count):
#         print('point 1')
        flag_start_count = True
        counter_zeros_in_sequence = 1
        left_index=i

fig = go.Figure()
# x = np.arange(len(prediction))
fig.add_trace(go.Scatter(x=train_data.index.values,
                         y=train_data['Value'],
                         name ='pre_interval')) 
    
    

In [0]:
# Making random not only in order?
def generate_random_batch(series, size=DAYS_PRED):      
    
    left_non_zero_index=0
    right_index=series.shape[0]-2*size    
    
    while series.iloc[left_non_zero_index, 1]==0:
        left_non_zero_index +=1

    r_i=np.random.randint(left_non_zero_index,right_index)
#     r_i=900 #for overfiting traingin
    batch_1=[]
    batch_2=[]
    
    batch_1=series.iloc[r_i:(r_i+size),1]
    batch_2=series.iloc[r_i+size:r_i+size*2,1]
    
    return batch_1, batch_2, r_i

# batch_1, batch_2, r_i=generate_random_batch(train_data)
# print(batch_1)
# print(batch_2)

In [0]:
def validation(model,display=False,display_figure=False):
#     prediction_2=[]
    y=[]
    summation=[]
#   for i in range(1300,1800,20):
#     for i in np.random.randint(train_data.index):  #need correction!!!
    for _ in range(5):
        i = np.random.randint(train_data.index.values.shape[0]-DAYS_PRED*2)  #need correction!!!  
#         i=900 #for overfiting traingin
        batch_test_1, batch_test_2, batch_true = test_batch_ri_from_train_data(train_data, r_i=i) 
        prediction=f28_days_prediction(model,batch_test_1,batch_test_2)
        y.append(rmsse(np.reshape(batch_true,28),prediction, display))
        summation.append(sum_delta(np.reshape(batch_true,28),prediction))   
    if display_figure:    
        fig = go.Figure()
        x = np.arange(len(y),len(prediction)+DAYS_PRED)
        fig.add_trace(go.Scatter(x=x, y=y, name ='tatata'))
        fig.show()
    if display:    
        print(f" summation: {sum(summation)}")
        print(f" RMSE: {sum(y)}")
    return sum(summation), sum(y)
   
#need correction!!!   
def optimization(model, batch_1,batch_2,display=False): 
    fun_value=np.empty(shape=[1,],dtype = float)
    for i in range(0,21):                         #need correction!!!
        batch_2[0,27,0]=i
        fun_value=np.append(fun_value,model([batch_1.astype(float), batch_2.astype(float)]).numpy())        
    fun_value=np.delete(fun_value,0,axis=0)
    i_min=np.argmin(fun_value)
    if display:
        print(f" function values: {fun_value} \n \
        min index (or index + 1 element): {i_min}  \n \
        function min value: {fun_value[i_min]}")    
    return fun_value[i_min], i_min
# _ , i_min  = optimization(model,batch_1,batch_2,display=True)
# print(i_min)

def f28_days_prediction(model, batch_1,batch_2):
    prediction=np.empty(shape=[1,],dtype = float)
    for i in range(28):    
        _ , new_prediction = optimization(model,batch_1,batch_2,display=False)
        prediction=np.append(prediction, new_prediction)
        batch_2[0,27,0]=new_prediction
        batch_1 = np.copy(batch_2) 
        batch_2 = np.roll(batch_2,-1)    

    prediction=np.delete(prediction,0,axis=0)
    return prediction

# prediction=f28_days_prediction(batch_1,batch_2)
# print(prediction)
# print(prediction.shape,type(prediction))
# fig = go.Figure()
# x = np.arange(len(prediction))
# fig.add_trace(go.Scatter(x=x, y=prediction))
# fig.show()

def test_batch_ri_from_train_data(series, size=DAYS_PRED,r_i = DAYS_PRED):      
    
    left_non_zero_index=0
#   right_index=series.shape[0]-2*size    
    right_index=r_i
    
    while series.iloc[left_non_zero_index, 1]==0:
        left_non_zero_index +=1
        
    if right_index<left_non_zero_index: right_index=left_non_zero_index   
       
    batch_1=series.iloc[r_i:(r_i+size),1]
    batch_2=series.iloc[r_i:(r_i+size),1]
    batch_true=series.iloc[(r_i+size+1):(r_i+1+size*2),1]
    
    batch_1 = tf.transpose(np.expand_dims(batch_1,axis=0))
    batch_2 = tf.transpose(np.expand_dims(batch_2,axis=0))
    batch_true = tf.transpose(np.expand_dims(batch_true,axis=0))
    
    batch_1 = np.expand_dims(batch_1,axis=0)
    batch_2 = np.expand_dims(batch_2,axis=0)
    batch_true = np.expand_dims(batch_true,axis=0)
    
    batch_2 = np.roll(batch_2,-1)
    
    return batch_1, batch_2, batch_true

# batch_1, batch_2, batch_true = test_batch_ri_from_train_data(train_data, size=DAYS_PRED,r_i = DAYS_PRED) 
# print(batch_2)

def rmsse(y_true, y_pred,display=False):
    if display:
        print(f"y_true sum: {np.sum(y_true)}, y_pred sum: {np.sum(y_pred)}")
    return np.sqrt(mean_squared_error(y_true, y_pred))

def sum_delta(y_true, y_pred):    
    return abs(np.sum(y_true)-np.sum(y_pred))


In [0]:
class Layer_MSE(layers.Layer):

    def __init__(self, units=32):
        super(Layer_MSE, self).__init__()
        
        
    def call(self, input1, input2):
#         print(input1.shape)
        out=tf.math.add(input1,-input2)
#         print(f"delta:{out}")
        out=out*out
#         print(f"out*out:{out}")
        out=tf.math.reduce_sum(out,axis=2)
#         print(f"reduce_sum:{out}")
        out = tf.math.sqrt(out)  
#       print(out)
        return out

# x1 = tf.constant([[[1.0, 1.0], [1.0, 1.0]],
#                    [[1.0, 1.0], [1.0, 1.0]],
#                    [[1.0, 1.0], [1.0, 1.0]]])
# x2 =  tf.constant([[[1.0, 2.0], [3.0, 4.0]],
#                    [[1.0, 2.0], [3.0, 4.0]],
#                    [[1.0, 2.0], [3.0, 4.0]]])
# print(x1.shape)

# test_layer = Layer_MSE()
# y = test_layer(x1,x2)
# print(y)

In [0]:
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# tf.keras.losses.Reduction.NONE

def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = myloss(model, inputs, targets, training=True) # True or False?
#         print(f"loss_value in grad fucntion: {loss_value}")
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

def training_loop(model):
    # Note: Rerunning this cell uses the same model variables
    global TRAIN_STAGE
    global BATCH_SIZE

    # Keep results for plotting
    train_loss_results = []
    train_accuracy_results = []
    
    optimizer = tf.keras.optimizers.Adam()
    print(optimizer)
    shift_for_train_stage=0
    for epoch in range(NUM_EPOCH):
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.Accuracy()

      # Training loop - using batches of 32        
#       for x,y in [[x_train, y_train]]:
        for _ in range(BATCH_SIZE):
            i=np.random.randint(len(y_train))            
            x = [np.expand_dims(x_train[0][i],axis=0), np.expand_dims(x_train[0][1],axis=0)]
            y = np.expand_dims(y_train[i],axis=0)
            
        # Optimize the model
            loss_value, grads = grad(model, x, y)
#             print(f"y value in training loop : {x}")
#             print(f"grads value in training loop : {grads}")
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            # Track progress
            epoch_loss_avg.update_state(loss_value)  # Add current batch loss
            # Compare predicted label to actual label
            # training=True is needed only if there are layers with different
            # behavior during training versus inference (e.g. Dropout).
            epoch_accuracy.update_state(y, model(x, training=True))

        # End epoch
        train_loss_results.append(epoch_loss_avg.result())
        train_accuracy_results.append(epoch_accuracy.result())

        #Save the model
        if (epoch % 10) == 0:
          save_path = r'/content/Saved models'
          name_of_file ="SIA_model_v12"
          complete_path = os.path.join(save_path,name_of_file)
          model.save(complete_path)
          print("Model Saved")

        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}, TRAIN_STAGE: {}".format(epoch,
                                                                    epoch_loss_avg.result(),
                                                                    epoch_accuracy.result(),
                                                                    TRAIN_STAGE))
        if (epoch-shift_for_train_stage)>10 and np.all(np.array(train_loss_results)[-10:]<=EARLY_STOP_LOSS_VALUE):
           print(f"------ New TRAIN_STAGE---------")
           TRAIN_STAGE +=1
           shift_for_train_stage=epoch
          #  BATCH_SIZE = 2
          #  optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.1)
           optimizer = tf.keras.optimizers.Adam()
           print(optimizer)
          

        if TRAIN_STAGE == 3:
            break
       
#         if epoch % 50 == 0:
#             print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
#                                                                     epoch_loss_avg.result(),
#                                                                     epoch_accuracy.result()))


# optimizer = tf.keras.optimizers.Adam()

# loss_value, grads = grad(model, features, labels)

# print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
#                                           loss_value.numpy()))

# optimizer.apply_gradients(zip(grads, model.trainable_variables))

# print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
#                                           loss(model, features, labels, training=True).numpy()))

In [0]:
# model2=create_model("test model")
# model2.summary()
# keras.utils.plot_model(model2,show_shapes=True)

In [0]:
def create_model(model_name=''):
    inputs_1 = keras.Input(shape=(DAYS_PRED,1))
    inputs_2 = keras.Input(shape=(DAYS_PRED,1))
#     BN = layers.BatchNormalization()
#     x_1 = BN(inputs_1)
#     x_2 = BN(inputs_2)    
    convlayer = layers.Conv1D(NUM_FILTERS_IN_CONV_LAYER,
                              KERNEL_SIZE,
                              padding='valid',
                              activation='sigmoid',
                              kernel_initializer=tf.random_uniform_initializer(0,1,seed=70)
                             )
#     x_1 = convlayer(x_1)
#     x_2 = convlayer(x_2)  
    x_1 = convlayer(inputs_1)
    x_2 = convlayer(inputs_2)
    x_1 = layers.AveragePooling1D(pool_size=DAYS_PRED-KERNEL_SIZE+1)(x_1)
    x_2 = layers.AveragePooling1D(pool_size=DAYS_PRED-KERNEL_SIZE+1)(x_2)
    out=Layer_MSE()(x_1,x_2)  
#     out=layers.Reshape(())(out)
   
    model = keras.Model([inputs_1,inputs_2], [out], name=model_name)
        
    return model
 
def train_model():
    
    model = create_model()
    keras.utils.plot_model(model,show_shapes=True)
    
#   loss_fn = tf.keras.losses.MeanAbsoluteError()
    loss_fn = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.NONE)
        
    training_loop(model)
    
#     model.compile(optimizer='adam',
#                 loss=Myloss,
#                 metrics=['accuracy'])
    
#     print('# Fit model on training data')
#     history=model.fit(x=x_train, 
#             y=y_train, 
#             epochs=30)
    
#     print('\nhistory dict:', history.history)
    return model


In [0]:
# model2=create_model()
#SAVE the model
# save_path = r'/content/Saved models'
# name_of_file ="SIA_model_v12"
# complete_path = os.path.join(save_path,name_of_file)
# model2.save(complete_path)

#Load the model
# new_model = tf.keras.models.load_model('/content/Saved models/SIA_model_v12')
# print(new_model.trainable_variables)


In [0]:
# sales, prices, calendar, submission = read_data()
# NUM_ITEMS = sales.shape[0]  # 30490
# DAYS_PRED = submission.shape[1] - 1  # 28

# Glogbal Parameters
NUM_ITEMS =  30490
DAYS_PRED =  28

# Global tuning variables
NUM_FILTERS_IN_CONV_LAYER=7
KERNEL_SIZE=14
NUM_EPOCH = 3000
BATCH_SIZE = 10
NUM_SAMPLES = 1000
EARLY_STOP_LOSS_VALUE=1

TRAIN_STAGE=0

def myloss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  # y_ = model(x, training=training)
        
    if TRAIN_STAGE == -1:
      y_ = model(x, training=True)
      RMSE=0
      summation=0
      return abs(y_)*10e2
    else:
      y_ = model(x, training=True)  
      summation, RMSE = validation(model)
      print(f"in myloss function: summation {summation}, RMSE: {RMSE}, TRAIN_STAGE: {TRAIN_STAGE}")
      return RMSE*10+abs(y_)

    
#     print(model.trainable_variables)     
#     print(f"y_ value in myloss function: {y_}, shape: {y_.shape}")
#     print(f"y value in myloss function: {y}, shape: {y.shape} ")
#     print(f"tf.reduce_mean(y_) value in myloss function: {tf.reduce_mean(y_)} ")

In [0]:
# START CALCULATION
input_1=np.empty(shape=[1,28,1],dtype = float)
input_2=np.empty(shape=[1,28,1],dtype = float)
for i in range(NUM_SAMPLES):
    batch_1, batch_2, _ = generate_random_batch(train_data)
#     batch_1, _, _ = generate_random_batch(train_data)
#     _, batch_2, _ = generate_random_batch(train_data)
    batch_1 = tf.transpose(np.expand_dims(batch_1,axis=0))
    batch_2 = tf.transpose(np.expand_dims(batch_2,axis=0))
    batch_1 = np.expand_dims(batch_1,axis=0)
    batch_2 = np.expand_dims(batch_2,axis=0)   
    input_1=np.append(input_1,batch_1,axis=0)
    input_2=np.append(input_2,batch_2,axis=0)

input_1=np.delete(input_1,0,axis=0)
input_2=np.delete(input_2,0,axis=0)

y_train=pd.Series(np.zeros(NUM_SAMPLES),dtype = float)
x_train = [input_1, input_2]

# print(f"y_train value in start : {y_train}")

# x_val=x_train
# y_val=y_train

TRAIN_STAGE=1
model=train_model()
keras.utils.plot_model(model,show_shapes=True)
# batch_1.shape
# model.evaluate([batch_1,batch_2])

# test1_model=create_model("test1 model")
# test1_model.summary()
# keras.utils.plot_model(test1_model,show_shapes=True)

Audio(filename="/content/Sounds/ring_end.mp3", autoplay=True)

in myloss function: summation 73.0, RMSE: 11.60626658488859, TRAIN_STAGE: 1
in myloss function: summation 128.0, RMSE: 17.6729429593413, TRAIN_STAGE: 1
in myloss function: summation 103.0, RMSE: 16.735141708507367, TRAIN_STAGE: 1
in myloss function: summation 139.0, RMSE: 19.034729630958758, TRAIN_STAGE: 1
in myloss function: summation 131.0, RMSE: 19.636204567463015, TRAIN_STAGE: 1
in myloss function: summation 158.0, RMSE: 20.474174307781098, TRAIN_STAGE: 1
in myloss function: summation 97.0, RMSE: 12.31214418670239, TRAIN_STAGE: 1
in myloss function: summation 86.0, RMSE: 14.809473678416252, TRAIN_STAGE: 1
in myloss function: summation 116.0, RMSE: 17.693759389403102, TRAIN_STAGE: 1
in myloss function: summation 103.0, RMSE: 15.88377923141715, TRAIN_STAGE: 1
INFO:tensorflow:Assets written to: /content/Saved models/SIA_model_v12/assets
Model Saved
Epoch 000: Loss: 165.930, Accuracy: 0.000%, TRAIN_STAGE: 1
in myloss function: summation 102.0, RMSE: 15.116157722754949, TRAIN_STAGE: 1
i

In [0]:
#Load the model
# load_model = tf.keras.models.load_model('/content/Saved models/SIA_model_v12')
# load_model.trainable_variables

In [0]:
def plot_x_train(x_train):    
    fig = go.Figure() 
    x = np.arange(x_train[1].shape[1])
    for i in range(len(x_train[1])):
        fig.add_trace(go.Scatter(x=x, y=np.reshape(x_train[0][i][:],28),name='0'))
        fig.add_trace(go.Scatter(x=x, y=np.reshape(x_train[1][i][:],28),name='1'))
    fig.show()
def plot_batch(batch):    
    fig = go.Figure()
    x = np.arange(batch.shape[1])
    fig.add_trace(go.Scatter(x=x, y=np.reshape(batch, 28),name='batch'))
    fig.show()

In [0]:
def last_batch_from_train_data(series, size=DAYS_PRED, x=0):      
    
    left_index=series.shape[0]-size   
    right_index=series.shape[0]    
       
    batch_1=series.iloc[left_index:right_index,1]
    batch_2=series.iloc[left_index:right_index,1]    
    
    batch_1 = tf.transpose(np.expand_dims(batch_1,axis=0))
    batch_2 = tf.transpose(np.expand_dims(batch_2,axis=0))
    batch_1 = np.expand_dims(batch_1,axis=0)
    batch_2 = np.expand_dims(batch_2,axis=0) 
    batch_2 = np.roll(batch_2,-1)
    
    return batch_1, batch_2
batch_1, batch_2 = last_batch_from_train_data(train_data)
batch_1 = np.copy(batch_1)
batch_2 = np.copy(batch_2)

# print(batch_1,batch_2)
# batch_2[0,27,0]=2
# print(model([batch_1.astype(float), batch_2.astype(float)]).numpy())

In [0]:
# plot_batch(batch_1)
# plot_batch(batch_2)
model2=create_model()
print(np.asarray(model2.trainable_variables)-np.asarray(model.trainable_variables))

In [50]:
#Load the model
if not 'model' in locals():
  model = tf.keras.models.load_model('/content/Saved models/SIA_model_v12')


# One test view on model working
batch_test_1, batch_test_2, batch_true = test_batch_ri_from_train_data(train_data, r_i=800)
prediction=f28_days_prediction(model,batch_test_1,batch_test_2)
fig = go.Figure()
x = np.arange(len(prediction))
fig.add_trace(go.Scatter(x=x, y=np.reshape(batch_test_1,28), name ='pre_interval'))
x = np.arange(len(prediction),len(prediction)+DAYS_PRED)
fig.add_trace(go.Scatter(x=x, y=prediction, name ='prediction'))
fig.add_trace(go.Scatter(x=x, y=np.reshape(batch_true,28), name ='true'))
fig.show()

 
print(f"model prediction: \n \
    {rmsse(np.reshape(batch_true,28),prediction)}")

print(f"ones line prediction: \n \
    {rmsse(np.reshape(batch_true,28),np.ones(28))}")

print(f" sum on true: {np.sum(np.reshape(batch_true,28))}, sum on prediction: {np.sum(prediction)}")

model prediction: 
     1.546886273407694
ones line prediction: 
     1.2955969390869324
 sum on true: 27, sum on prediction: 18.0


In [52]:
#Load the model
if not 'model' in locals():
  model = tf.keras.models.load_model('/content/Saved models/SIA_model_v12')

test_summation=[]
test_rmse=[]

for _ in range(5):
    summation, rmse = validation(model,display=True)
    test_summation.append(summation)
    test_rmse.append(rmse)
print(f"test summation: {sum(test_summation)}, test rmse: {sum(test_rmse)}")

y_true sum: 21, y_pred sum: 18.0
y_true sum: 19, y_pred sum: 19.0
y_true sum: 11, y_pred sum: 10.0
y_true sum: 14, y_pred sum: 24.0
y_true sum: 25, y_pred sum: 22.0
 summation: 17.0
 RMSE: 7.0930807214873415
y_true sum: 25, y_pred sum: 23.0
y_true sum: 14, y_pred sum: 16.0
y_true sum: 18, y_pred sum: 23.0
y_true sum: 20, y_pred sum: 22.0
y_true sum: 20, y_pred sum: 21.0
 summation: 12.0
 RMSE: 8.761575179739424
y_true sum: 23, y_pred sum: 22.0
y_true sum: 22, y_pred sum: 23.0
y_true sum: 14, y_pred sum: 17.0
y_true sum: 15, y_pred sum: 19.0
y_true sum: 17, y_pred sum: 12.0
 summation: 14.0
 RMSE: 5.928721084547755
y_true sum: 8, y_pred sum: 21.0
y_true sum: 28, y_pred sum: 21.0
y_true sum: 11, y_pred sum: 14.0
y_true sum: 17, y_pred sum: 19.0
y_true sum: 21, y_pred sum: 23.0
 summation: 27.0
 RMSE: 7.358995456264516
y_true sum: 8, y_pred sum: 19.0
y_true sum: 13, y_pred sum: 21.0
y_true sum: 22, y_pred sum: 19.0
y_true sum: 24, y_pred sum: 23.0
y_true sum: 8, y_pred sum: 19.0
 summatio

In [0]:
# i = np.random.choice(train_data.index.values)
# print(train_data.index.values.shape)
# print(i)
# i=1835
# test_batch_ri_from_train_data(train_data, r_i=900)